In [2]:
import requests                 # Библиотека работы с HTTP-запросами по API
import json                     # Для обработки полученных результатов запросов
import time                     # Для задержки между запросами
import os                       # Для работы с файлами
import pandas as pd             # Для формирования датафрейма с результатами
import re                       # Для работы с регулярными выражениями
import numpy as np              # Библиотека работы с массивами
import seaborn as sns           # Статистическая визуализация данных
import matplotlib.pyplot as plt # Визуализация данных
import plotly.graph_objs as go
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords


import requests
from PIL import Image
from io import BytesIO


In [20]:
import requests

client_id = '<client_id>'
client_secret = '<client_secret>'
redirect_uri = '<redirect_uri>'
authorization_code = '<authorization_code>'

url = 'https://hh.ru/oauth/token'
params = {
    'grant_type': 'client_credentials',
    'client_id': 'QVQPMTPM4PUQRILS8IKA5HFSCCUTCF6ALDVF2UMNLTESP1BLJ9PDC91AHFNLR140',
    'client_secret': 'QQOHNMADB540ANAD61SJRJ4A29FK9N49HCA13QTV9M7E5QUQDRMJBG6J051FU487',
    'redirect_uri': 'https://hh.ru',
}
response = requests.post(url, params=params)

access_token = response.json().get('access_token')

response


<Response [200]>

In [19]:
response

<Response [403]>

In [47]:
access_token = 'I4USFCV13SCNQ3TKKAD3R6MN4ODP8FBH5TMJSV05JMSMCHDOMBGV7VB8OQ35VL65'

In [51]:
def get_vacancies(job_title, access_token, page=0, retries=5):
    '''Функция запроса страницы.
       job_title - наименование вакансии
       access_token - токен авторизации
       page - Индекс страницы
       retries - количество попыток повторения запроса
       (по умолчанию начинается с первой страницы)'''

    # Справочник для переменной GET-запроса
    # Значения для указания значений в справочнике 
    # указаны в документации к API hh.ru по ссылке: 
    # https://github.com/hhru/api
    headers = {'Authorization': f'Bearer I4USFCV13SCNQ3TKKAD3R6MN4ODP8FBH5TMJSV05JMSMCHDOMBGV7VB8OQ35VL65'}
    params = {
        'text': job_title,
        'area': 113,
        'page': page,
        'per_page': 100
    }

    for i in range(retries):
        try:
            req = requests.get('https://api.hh.ru/vacancies', params=params, headers=headers, timeout=20)
            data = req.content.decode()
            req.close()
            response = json.loads(data)
            # Проверяем, что в ответе нет ошибки Bad Gateway
            if 'errors' in response and any(error['type'] == 'bad_gateway' for error in response['errors']):
                print(f'Получена ошибка Bad Gateway. Повторяем через {3 * (2 ** i)} сек.')
                if i == retries - 1:
                    print('Не удалось выполнить запрос. Проверьте соединение.')
                    return None
                time.sleep(3 * (2 ** i))
                continue
            return data
        except requests.exceptions.RequestException as e:
            print(f"Ошибка при запросе страницы {page}: {e}")
            print(f"Попытка {i+1}/{retries}...")
            time.sleep(0.25)

    print(f"Не удалось получить данные для страницы {page}")
    return None

In [54]:
# Сделаем запрос к API hh.ru по вакансиям

vacancies = []
vacancies_ids = set()
job_titles = [
    "data engineer",
    "инженер данных",
    "data analyst",
    "аналитик данных",
    "аналитик",
    "analyst",
    "data",
    "дата",
    "data scientist",
    "сайнтист",
    "сайнс"
    "BI",
    "ETL"
]

for job_title in job_titles:
    for page in range(0, 100):
        # Запрашиваем страницу с вакансиями
        response = json.loads(get_vacancies(job_title, page))
        # Получаем список вакансий на странице
        items = response['items']
        for item in items:
            # Проверяем, что вакансия не была уже добавлена в список
            if item['id'] not in vacancies_ids:
                # Проверяем, что в названии вакансии есть искомое слово
                if any(title in item['name'].lower() for title in job_titles):
                    # Добавляем вакансию в список
                    vacancies.append(item)
                    # Добавляем id вакансии во множество уже просмотренных id
                    vacancies_ids.add(item['id'])
        # Проверяем, является ли страница последней
        if response['pages'] - page <= 1:
            break
        # # Делаем задержку между запросами
        # time.sleep(0.25)
if not os.path.exists('data'):
    os.makedirs('data')

In [55]:
# Сохраняем полученные вакансии в файле
with open('data/vacancies.json', 'w', encoding='utf-8') as f:
    f.write('')

print('Вакансии по запросу собраны и сохранены в файл vacancies.json')
# Количество вакансий
print(f"Количество собранных вакансий: {len(vacancies)}")

# Количество уникальных значений ключа 'name'
unique_names = set([v['name'] for v in vacancies])
names_count = len(unique_names)

print(f"Количество уникальных названий вакансий: {names_count}")

Вакансии по запросу собраны и сохранены в файл vacancies.json
Количество собранных вакансий: 481
Количество уникальных названий вакансий: 302


In [84]:
response

NameError: name 'response' is not defined

In [97]:
def get_skills(vacancy_id, token):
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'
    headers = {'User-Agent': 'Mozilla/5.0', 'Authorization': f'Bearer {token}'}
    skills = ""
    languages = ""
    while True:
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                data = response.json()
                skills = data['key_skills']
                languages = data['languages']
                # time.sleep(0.5)
                break
            elif response.status_code == 403:
                raise ValueError("Требуется решить капчу. Прекращение выполнения функции.")
        except requests.exceptions.RequestException:
            print(f"Ошибка получения описания вакансии {vacancy_id}. Повтор запроса через 0.2 секунды.")
            time.sleep(0.2)
            continue
    
    return skills, languages


In [99]:
vacancy_id = '79190614'
token = 'I4USFCV13SCNQ3TKKAD3R6MN4ODP8FBH5TMJSV05JMSMCHDOMBGV7VB8OQ35VL65'
skills = get_skills(vacancy_id, token)

In [100]:
skills

([],
 [{'id': 'eng',
   'name': 'Английский',
   'level': {'id': 'b2', 'name': 'B2 — Средне-продвинутый'}}])

In [104]:
# Чтение файла vacancies.json и создание словаря skills_dict
with open('data/vacancies.json', 'r', encoding='utf-8') as f:
    vacancies = json.load(f)
skills_dict = {vacancy["id"]: "" for vacancy in vacancies}

In [107]:
# Получим незагруженные скилы

# Обход словаря skills_dict и заполнение значениями ключа "key_skills"
# делаем это для вакансий, описание которых еще не получили 

def fill_vacancy_skills(skills_dict, token):
    temp_dict = {}
    for id, skills in skills_dict.items():
        if not skills:
            try:
                new_skill, languages = get_skills(id, token)
                if new_skill:
                    temp_dict[id] = {'key_skills': new_skill, 'languages': languages}
                    # time.sleep(0.2)
            except ValueError as e:
                print(e)
                break
    
    # Изменяем исходный словарь после завершения цикла итерации
    for id, skills in temp_dict.items():
        skills_dict[id] = skills['key_skills']
        # Для languages может быть отдельный словарь, если он нужен
    
    return skills_dict


In [57]:
# МОДУЛЬ ДОЗАПОЛНЕНИЯ

# # Чтение файла vacancies_with_descriptions.json и создание словаря vacancies_dict
# with open('data/vacancies_with_descriptions.json', 'r', encoding='utf-8') as f:
#     vacancies = json.load(f)
# vacancies_dict = {vacancy["id"]: vacancy["description"] for vacancy in vacancies}

# заполняем описание вакансий, у которых в ключе skills пусто 

token = 'I4USFCV13SCNQ3TKKAD3R6MN4ODP8FBH5TMJSV05JMSMCHDOMBGV7VB8OQ35VL65'
def fill_vacancy_skills(skills_dict, token):
    for id, skills in skills_dict.items():
        if not skills:
            try:
                new_skill = get_skills(id, token)
                if new_skill:
                    skills_dict[id] = new_skill
                    # time.sleep(0.2)
            except ValueError as e:
                print(e)
                break
        else:
            new_skill = []
    return skills_dict

filled_dict = fill_vacancy_skills(skills_dict, token)


KeyboardInterrupt: 

In [105]:
### ДОЗАПОЛНЕНИЕ
def fill_vacancy_skills(skills_dict, token):
    for vacancy_id, skill in skills_dict.items():
        if not skill:
            print('get for', vacancy_id)
            try:
                new_skill = get_skills(vacancy_id, token)
                print(new_skill)
                if new_skill:
                    skills_dict[id] = new_skill
            except ValueError as e:
                print(e)
                break
    return skills_dict


In [102]:
get_skills(79190614,token)

([],
 [{'id': 'eng',
   'name': 'Английский',
   'level': {'id': 'b2', 'name': 'B2 — Средне-продвинутый'}}])

In [89]:
### ПОЛНОЕ ОПИСАНИЕ
def get_vacancy_description(vacancy_id, token):
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'
    headers = {'User-Agent': 'Mozilla/5.0', 'Authorization': f'Bearer {token}'}
    description = ""
    
    while True:
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                data = response.json()
                description = data
                time.sleep(0.5)
                break
            elif response.status_code == 403:
                raise ValueError("Требуется решить капчу. Прекращение выполнения функции.")
        except requests.exceptions.RequestException:
            print(f"Ошибка получения описания вакансии {vacancy_id}. Повтор запроса через 0.2 секунды.")
            time.sleep(0.2)
            continue
    
    return description


In [90]:
get_vacancy_description(79190614,token)

{'id': '79190614',
 'premium': False,
 'billing_type': {'id': 'standard', 'name': 'Стандарт'},
 'relations': [],
 'name': 'Data Engineering Intern',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': {'from': 85000, 'to': None, 'currency': 'RUR', 'gross': True},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Москва',
  'street': 'Ленинградское шоссе',
  'building': '16ас2',
  'lat': 55.821573,
  'lng': 37.498725,
  'description': None,
  'raw': 'Москва, Ленинградское шоссе, 16ас2',
  'metro': {'station_name': 'Войковская',
   'line_name': 'Замоскворецкая',
   'station_id': '2.30',
   'line_id': '2',
   'lat': 55.818923,
   'lng': 37.497791},
  'metro_stations': [{'station_name': 'Войковская',
    'line_name': 'Замоскворецкая',
    'station_id': '2.30',
    'line_id': '2',
    'lat': 55.818923,
    'lng': 37.497791}]},
 'allow_messages': False,
 'experience': {'id': 'n

In [111]:
filled_dict = fill_vacancy_skills(skills_dict, token)


Ошибка получения описания вакансии 78996792. Повтор запроса через 0.2 секунды.
Ошибка получения описания вакансии 78936789. Повтор запроса через 0.2 секунды.


KeyboardInterrupt: 

In [112]:
# счетчики в переменной filled_dict

total_vacancies = len(filled_skills)
empty_vacancies = sum(1 for desc in filled_skills.values() if not desc)
filled_in_filled = sum(1 for desc in ffilled_skills.values() if desc)
print(f'total: {total_vacancies}')
print(f'filled: {filled_in_filled}')
print(f'empty: {empty_vacancies}')

NameError: name 'filled_skills' is not defined

In [114]:
# счетчики в переменной skills_dict

total_vacancies = len(skills_dict)
empty_vacancies = sum(1 for desc in skills_dict.values() if not desc)
filled_vacancies = sum(1 for desc in skills_dict.values() if desc)
print(f'total: {total_vacancies}')
print(f'filled: {filled_vacancies}')
print(f'empty: {empty_vacancies}')

total: 5311
filled: 1
empty: 5310


In [115]:
skills_dict

{'79110745': '',
 '79120615': '',
 '78934984': '',
 '78954091': '',
 '79084256': '',
 '78613909': '',
 '79315237': '',
 '79192759': '',
 '79323507': '',
 '79172870': '',
 '78968207': '',
 '78370985': '',
 '78340750': '',
 '79190614': '',
 '79226963': '',
 '78448054': '',
 '78952166': '',
 '78687348': '',
 '78879859': '',
 '77577033': '',
 '78273747': '',
 '79261592': '',
 '79148004': '',
 '79145131': '',
 '78029161': '',
 '79162833': '',
 '79226964': '',
 '78427104': '',
 '79115441': '',
 '78519105': '',
 '78852846': '',
 '79305476': '',
 '79314682': '',
 '78258371': '',
 '79122630': '',
 '79089912': '',
 '79314135': '',
 '79147234': '',
 '77426640': '',
 '79127417': '',
 '78664757': '',
 '71220088': '',
 '79232504': '',
 '79314029': '',
 '78149780': '',
 '78386771': '',
 '78879858': '',
 '79279390': '',
 '79221567': '',
 '78306430': '',
 '79300938': '',
 '79240771': '',
 '73888449': '',
 '79224520': '',
 '79090742': '',
 '78617302': '',
 '79267194': '',
 '78371688': '',
 '79177815': '

НИЖЕ ПРИВЕДЕНИЕ НАЗВАНИЙ ВАКАНСИЙ К 5 ОСНОВНЫМ


In [ ]:
# Очистим от грейдов
# Список слов для удаления
words_to_remove = [
        'Junior' 
        , 'junior' 
        , 'Младший'
        , 'младший'
        , 'Senior' 
        , 'senior' 
        , 'Старший' 
        , 'старший' 
        , 'Ведущий' 
        , 'ведущий' 
        , 'Middle'
        , 'middle'
            
]
    
# Создаем регулярное выражение для удаления указанных слов
pattern = '|'.join(words_to_remove)
    
# Применяем регулярное выражение к столбцу 'name'
df['name_clean'] = df['name'].apply(lambda x: re.sub(pattern, '', x))

# Удаление пробелов в начале и конце столбца 'name'
df['name_clean'] = df['name_clean'].str.strip()

In [ ]:
# Вновь посмотрим топ-30 названий вакансий
counts = df['name_clean'].value_counts()
top_30 = counts.head(30)
print(top_30)

name_clean
Аналитик                               431
Бизнес-аналитик                        252
Системный аналитик                     183
Аналитик данных                        143
Data Engineer                          112
Маркетолог-аналитик                    112
Финансовый аналитик                     86
аналитик                                86
Экономист-аналитик                      84
Data Scientist                          68
Аналитик отдела продаж                  44
Data Analyst                            41
Продуктовый аналитик                    38
Аналитик BI                             33
Системный аналитик DWH                  30
Data engineer                           29
Аналитик бизнес-процессов               26
бизнес-аналитик                         23
Аналитик DWH                            20
Product Analyst                         20
Business Data Engineer                  19
системный аналитик                      18
System Analyst                          16


In [ ]:
vacancies_dict = {
    "BI - Data Analyst ()": "Business Intelligence (BI) Analyst",
    "BI Data Analyst": "Business Intelligence (BI) Analyst",
    "BI analyst (Visiology)": "Business Intelligence (BI) Analyst",
    "BI Аналитик": "Business Intelligence (BI) Analyst",
    "BI аналитик": "Business Intelligence (BI) Analyst",
    "BI-Data Analyst": "Business Intelligence (BI) Analyst",
    "BI-аналитик": "Business Intelligence (BI) Analyst",
    "Business Analyst": "Business Intelligence (BI) Analyst",
    "Business Intelligence (BI) Analyst (удаленно)": "Business Intelligence (BI) Analyst",
    "Business Intelligence (BI) Analyst в отчетность DWH": "Business Intelligence (BI) Analyst",
    "Business/System Analyst (DWH)": "Business Intelligence (BI) Analyst",
    "Data Analyst BI": "Business Intelligence (BI) Analyst",
    "Data Analyst BI (отчетность DWH)": "Business Intelligence (BI) Analyst",
    "Data Analyst бизнес-процессов": "Business Intelligence (BI) Analyst",
    "Product Analyst": "Business Intelligence (BI) Analyst",
    "Product analyst": "Business Intelligence (BI) Analyst",
    "Аналитик BI": "Business Intelligence (BI) Analyst",
    "Аналитик BI (отчетность DWH)": "Business Intelligence (BI) Analyst",
    "Аналитик DWH": "Business Intelligence (BI) Analyst",
    "Аналитик бизнес-процессов": "Business Intelligence (BI) Analyst",
    "Аналитик отдела продаж": "Business Intelligence (BI) Analyst",
    "Бизнес-аналитик": "Business Intelligence (BI) Analyst",
    "Бизнес-аналитик в отчетность DWH": "Business Intelligence (BI) Analyst",
    "Руководитель Business Intelligence (BI) Analystи (г. Казань)": "Business Intelligence (BI) Analyst",
    "Специалист отдела методологии (Business Intelligence (BI) Analyst)": "Business Intelligence (BI) Analyst",
    "бизнес-аналитик": "Business Intelligence (BI) Analyst",
    "Analyst (remote)": "Data Analyst",
    "CRM Analyst": "Data Analyst",
    "Data Analyst": "Data Analyst",
    "Data Analyst ()": "Data Analyst",
    "Data Analyst (удаленно)": "Data Analyst",
    "Data Analyst / Data Analyst данных": "Data Analyst",
    "Data Analyst / Медиа-Data Analyst": "Data Analyst",
    "Data Analyst CRM": "Data Analyst",
    "Data Analyst Power BI": "Data Analyst",
    "Data Analyst SQL": "Data Analyst",
    "Data Analyst баз данных": "Data Analyst",
    "Data Analyst в области информационных систем": "Data Analyst",
    "Data Analyst в отдел маркетинга": "Data Analyst",
    "Data Analyst данных": "Data Analyst",
    "Data Analyst данных (/+)": "Data Analyst",
    "Data Analyst данных (Data Analyst)": "Data Analyst",
    "Data Analyst данных (SQL)": "Data Analyst",
    "Data Analyst данных (Владимир)": "Data Analyst",
    "Data Analyst данных / Data Analyst": "Data Analyst",
    "Data Analyst данных CV": "Data Analyst",
    "Data Analyst клиентских данных": "Data Analyst",
    "Data Analyst маркетплейсов": "Data Analyst",
    "Data Analyst отдела закупок": "Data Analyst",
    "Data Analyst отдела маркетинга": "Data Analyst",
    "Data Analyst отдела продаж": "Data Analyst",
    "Data Analyst отчётности и баз данных": "Data Analyst",
    "Data Analyst по закупкам": "Data Analyst",
    "Data Analyst по планированию": "Data Analyst",
    "Data Analyst по продажам": "Data Analyst",
    "Data Analyst по ценообразованию": "Data Analyst",
    "Data Analyst пресейла": "Data Analyst",
    "Data Analyst продаж": "Data Analyst",
    "Data Analyst-маркетолог": "Data Analyst",
    "Data analyst": "Data Analyst",
    "Data-Data Analyst": "Data Analyst",
    "Marketing analyst": "Data Analyst",
    "System Analyst / Системный Data Analyst": "Data Analyst",
    "System Analyst \\ Системный Data Analyst": "Data Analyst",
    "System Analyst \\ Системный Аналитик 1": "Data Analyst",
    "System Analyst/Системный Data Analyst": "Data Analyst",
    "Web-Data Analyst": "Data Analyst",
    "Web-аналитик": "Data Analyst",
    "Аналитик": "Data Analyst",
    "Аналитик ()": "Data Analyst",
    "Аналитик CRM": "Data Analyst",
    "Аналитик SQL": "Data Analyst",
    "Аналитик в отдел маркетинга": "Data Analyst",
    "Аналитик данных": "Data Analyst",
    "Аналитик данных (/+)": "Data Analyst",
    "Аналитик данных (Владимир)": "Data Analyst",
    "Аналитик данных / Data Analyst": "Data Analyst",
    "Аналитик маркетплейсов": "Data Analyst",
    "Аналитик отдела маркетинга": "Data Analyst",
    "Аналитик по продажам": "Data Analyst",
    "Аналитик по ценообразованию": "Data Analyst",
    "Аналитик пресейла": "Data Analyst",
    "Аналитик продаж": "Data Analyst",
    "Аналитик-маркетолог": "Data Analyst",
    "Бизнес Data Analyst": "Data Analyst",
    "Веб-Data Analyst": "Data Analyst",
    "Веб-аналитик": "Data Analyst",
    "Инвестиционный Data Analyst": "Data Analyst",
    "Маркетинговый Data Analyst": "Data Analyst",
    "Маркетолог-Data Analyst": "Data Analyst",
    "Маркетолог-аналитик": "Data Analyst",
    "Менеджер-Data Analyst": "Data Analyst",
    "Менеджер-аналитик": "Data Analyst",
    "Оператор-Data Analyst": "Data Analyst",
    "Помощник Data Analystа": "Data Analyst",
    "Помощник аналитика": "Data Analyst",
    "Продуктовый Data Analyst": "Data Analyst",
    "Продуктовый аналитик": "Data Analyst",
    "Продуктовый аналитик /middl+ (удаленно)": "Data Analyst", 
    "Риск-Data Analyst": "Data Analyst",
    "Риск-аналитик": "Data Analyst",
    "Руководитель отдела Data Analystи": "Data Analyst",
    "Системный Data Analyst": "Data Analyst",
    "Системный Data Analyst ()": "Data Analyst",
    "Системный Data Analyst (удаленно)": "Data Analyst",
    "Системный Data Analyst / System Analyst": "Data Analyst",
    "Системный Data Analyst / system analyst": "Data Analyst",
    "Системный Data Analyst в команду Statistic & Marketing": "Data Analyst",
    "Системный аналитик": "Data Analyst",
    "Специалист отдела Data Analystи": "Data Analyst",
    "Специалист-Data Analyst": "Data Analyst",
    "Специалист-аналитик": "Data Analyst",
    "Стажер-Data Analyst": "Data Analyst",
    "Стажер-аналитик": "Data Analyst",
    "Товарный Data Analyst": "Data Analyst",
    "Товарный аналитик": "Data Analyst",
    "Трейдер-Data Analyst": "Data Analyst",
    "Трейдер-аналитик": "Data Analyst",
    "Финансист-Data Analyst": "Data Analyst",
    "Финансовый Data Analyst": "Data Analyst",
    "Финансовый аналитик": "Data Analyst",
    "Финансовый менеджер-Data Analyst": "Data Analyst",
    "Финансовый менеджер-аналитик": "Data Analyst",
    "Экономист-Data Analyst": "Data Analyst",
    "Экономист-аналитик": "Data Analyst",
    "Эксперт-Data Analyst": "Data Analyst",
    "аналитик": "Data Analyst",
    "аналитик данных": "Data Analyst",
    "маркетолог-Data Analyst": "Data Analyst",
    "маркетолог-аналитик": "Data Analyst",
    "финансовый Data Analyst": "Data Analyst",
    "финансовый аналитик": "Data Analyst",
    "/ Data Engineer": "Data Engineer",
    "Business Data Engineer": "Data Engineer",
    "Data Engineer": "Data Engineer",
    "Data Engineer ()": "Data Engineer",
    "Data Engineer (DWH)": "Data Engineer",
    "Data Engineer (ДАДМ)": "Data Engineer",
    "Data Engineer / Инженер данных": "Data Engineer",
    "Data engineer": "Data Engineer",
    "Data инженер": "Data Engineer",
    "Data-инженер": "Data Engineer",
    "System Analyst": "Data Engineer",
    "Дата-инженер": "Data Engineer",
    "Инженер данных": "Data Engineer",
    "Инженер-Data Analyst": "Data Engineer",
    "Data Engineer / Дата-инженер ()"
    "Инженер-Data Analyst (IT)": "Data Engineer",
    "Оператор-аналитик": "Data Engineer",
    "Системный аналитик ()": "Data Engineer",
    "Системный аналитик (удаленно)": "Data Engineer",
    "/ Data Scientist": "Data Scientist",
    "Data Scientist": "Data Scientist",
    "Data Scientist ": "Data Scientist",
    "Data Scientist (+/)": "Data Scientist",
    "Data Scientist (NLP)": "Data Scientist",
    "Data Scientist()": "Data Scientist",
    "Data Scientist(+/)": "Data Scientist",
    "Data Scientist(NLP)": "Data Scientist",
    "Data Scientist(Северсталь Диджитал)": "Data Scientist",
    "Data scientist": "Data Scientist",
    "Инвестиционный аналитик": "Data Scientist",
    "Data Analyst DWH": "Database Administrator (DBA)",
    "Database Admin": "Database Administrator (DBA)",
    "Database Administrator": "Database Administrator (DBA)",
    "Database administrator": "Database Administrator (DBA)",
    "System Analyst Trainee": "Database Administrator (DBA)",
    "System analyst": "Database Administrator (DBA)",
    "Администратор баз данных": "Database Administrator (DBA)",
    "Системный аналитик DWH": "Database Administrator (DBA)",
    "Специалист по базам данных": "Database Administrator (DBA)",
    "системный аналитик": "Database Administrator (DBA)",
}


In [ ]:
# Удаление пробелов в начале и конце столбца 'name_clean'
df['name_clean'] = df['name_clean'].str.strip()

# Замена значений в соответствии со словарем только при точном совпадении
df['name_clean'] = df['name_clean'].replace(vacancies_dict)


In [ ]:
# vacancies_dict.update(
#     {
#         "System Analyst": "Data Engineer",
#         "Маркетолог-аналитик": "Data Analyst",
#         "Финансовый аналитик": "Data Analyst",
#         "Экономист-аналитик": "Data Analyst",
#         "Продуктовый аналитик": "Data Analyst",
#         "Аналитик DWH": "Business Intelligence (BI) Analyst",
#         "Веб-аналитик": "Data Analyst",
#         "аналитик данных": "Data Analyst",
#         "BI Аналитик": "Business Intelligence (BI) Analyst",
#         "BI аналитик": "Business Intelligence (BI) Analyst",
#         "Аналитик SQL": "Data Analyst",
#         "Аналитик продаж": "Data Analyst",
#         "BI-аналитик": "Business Intelligence (BI) Analyst",
#         "Инвестиционный аналитик": "Data Scientist",
#         "Аналитик-маркетолог": "Data Analyst",
#         "финансовый аналитик": "Data Analyst",
#         "Аналитик маркетплейсов": "Data Analyst",
#         "Стажер-аналитик": "Data Analyst",
#         "Бизнес-аналитик в отчетность DWH": "Business Intelligence (BI) Analyst",
#         "Аналитик BI (отчетность DWH)": "Business Intelligence (BI) Analyst",
#         "Товарный аналитик": "Data Analyst",
#         "Аналитик данных (/+)": "Data Analyst",
#         "Риск-аналитик": "Data Analyst",
#         "Data Scientist (+/)": "Data Scientist",
#         "Web-аналитик": "Data Analyst",
#         "Помощник аналитика": "Data Analyst",
#         "Менеджер-аналитик": "Data Analyst",
#         "Аналитик по продажам": "Data Analyst",
#         "Аналитик в отдел маркетинга": "Data Analyst",
#         "Аналитик данных / Data Analyst": "Data Analyst",
#         "Аналитик пресейла": "Data Analyst",
#         "Трейдер-аналитик": "Data Analyst",
#         "Аналитик CRM": "Data Analyst",
#         "Финансовый менеджер-аналитик": "Data Analyst",
#         "Data Scientist (NLP)": "Data Scientist",
#         "Оператор-аналитик": "Data Engineer",
#         "Аналитик ()": "Data Analyst",
#         "Аналитик по ценообразованию": "Data Analyst",
#         "Аналитик отдела маркетинга": "Data Analyst",
#         "Системный аналитик (удаленно)": "Data Engineer",
#         "Аналитик данных (Владимир)": "Data Analyst",
#         "Системный аналитик ()": "Data Engineer",
#         "маркетолог-аналитик": "Data Analyst",
#         "Специалист-аналитик": "Data Analyst",
#     }
# )

# vacancies_dict = dict(sorted(vacancies_dict.items(), key=lambda x: x[1]))


In [ ]:
vacancies_dict

{'BI - Data Analyst ()': 'Business Intelligence (BI) Analyst',
 'BI Data Analyst': 'Business Intelligence (BI) Analyst',
 'BI analyst (Visiology)': 'Business Intelligence (BI) Analyst',
 'BI Аналитик': 'Business Intelligence (BI) Analyst',
 'BI аналитик': 'Business Intelligence (BI) Analyst',
 'BI-Data Analyst': 'Business Intelligence (BI) Analyst',
 'BI-аналитик': 'Business Intelligence (BI) Analyst',
 'Business Analyst': 'Business Intelligence (BI) Analyst',
 'Business Intelligence (BI) Analyst (удаленно)': 'Business Intelligence (BI) Analyst',
 'Business Intelligence (BI) Analyst в отчетность DWH': 'Business Intelligence (BI) Analyst',
 'Business/System Analyst (DWH)': 'Business Intelligence (BI) Analyst',
 'Data Analyst BI': 'Business Intelligence (BI) Analyst',
 'Data Analyst BI (отчетность DWH)': 'Business Intelligence (BI) Analyst',
 'Data Analyst бизнес-процессов': 'Business Intelligence (BI) Analyst',
 'Product Analyst': 'Business Intelligence (BI) Analyst',
 'Product analyst'

In [ ]:
counts = df['name_clean'].value_counts().sort_values(ascending=False).head(50)
counts_df = counts.to_frame().reset_index().rename(columns={'index': 'name_clean', 'name_clean': 'count'})
counts_df

,count,count
0,Data Analyst,727
1,Business Intelligence (BI) Analyst,436
2,Data Engineer,215
3,Системный аналитик,183
4,Маркетолог-аналитик,112
5,Финансовый аналитик,86
6,Экономист-аналитик,84
7,Data Scientist,81
8,Database Administrator (DBA),78
9,Продуктовый аналитик,38


In [ ]:
df['name_clean'].unique()

array(['Data Engineer', 'Data engineer (Стажер)',
       'Data Engineer / Дата-инженер ()', ...,
       'бизнес-аналитик (Служба стратегии и развития)',
       'Системный аналитик управления противодействия кибермошенничеству',
       'Аналитик систем целевого маркетинга'], dtype=object)

In [ ]:
# Вновь посмотрим топ-50 названий вакансий
counts = df['name_clean'].value_counts()
top_50 = counts.head(50)
top_50

name_clean
Data Analyst                           727
Business Intelligence (BI) Analyst     436
Data Engineer                          215
Системный аналитик                     183
Маркетолог-аналитик                    112
Финансовый аналитик                     86
Экономист-аналитик                      84
Data Scientist                          81
Database Administrator (DBA)            78
Продуктовый аналитик                    38
Аналитик DWH                            20
Веб-аналитик                            15
System Analyst \ Системный Аналитик     14
аналитик данных                         13
BI Аналитик                             12
BI аналитик                             12
Аналитик SQL                            12
Аналитик продаж                         10
BI-аналитик                             10
Инвестиционный аналитик                  9
Аналитик-маркетолог                      9
финансовый аналитик                      9
Аналитик маркетплейсов                   9


НИЖЕ КАКАЯ_ТО ШЛЯПА


In [ ]:
# определим 
def top_keys(df):
    # Collect keys from dataframe
    keys_df = df['description'].to_list()
    
    # Create a list of keys for all vacancies
    lst_keys = []
    for keys_elem in keys_df:
        for el in keys_elem[1:-1].split(', '):
            if el != '':
                lst_keys.append(re.sub('\'', '', el.lower()))

    # Unique keys and their counter
    set_keys = set(lst_keys)
    # Dict: {Key: Count}
    dct_keys = {el:lst_keys.count(el) for el in set_keys}
    # Sorted dict
    srt_keys = dict(sorted(dct_keys.items(), key=lambda x: x[1], reverse=True))
    # Return pandas series
    return pd.Series(srt_keys, name='Keys')

ds_keys = top_keys(df)

print(ds_keys[:30])

In [ ]:
print(ds_keys[:30])

In [ ]:
df.loc[
    df["description"].str.contains("kafka")]

In [ ]:
# Выведем топ-50 слов, встречающихся в описаниях
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# объединяем все описания в одну строку
text = ' '.join(df['description_preprocessed'].astype(str).tolist())

# токенизируем текст на слова
tokens = word_tokenize(text)

# создаем частотный словарь
freq_dist = FreqDist(tokens)

# выводим топ-50 слов
top_words = freq_dist.most_common(100)
print(top_words)


Зарплаты указаны в 20% вакансий. Попробуем предсказать зарплаты методом машиннного обучения. Для этого нам необходимо заполнить пропущенные значения вакансий, перевести все зарплаты в рубли и вычесть налог там, где вознаграждение указано гросс 

In [ ]:
df.head(3)

In [ ]:
# среднее значение столбцов salary.from и salary.to
mean_salary = (df['salary.from'] + df['salary.to']) / 2

# умножаем на курс валют (по данным ЦБ РФ
# https://www.cbr.ru/currency_base/daily/)
# и находим медианную зарплату

currency_coef = df['salary.currency'].map({'EUR': 89.5, 'USD': 81.6, 'RUR': 1})
mean_salary = mean_salary * currency_coef

# вычетаем подоходный налог, если значение в столбце salary.gross равно True
mean_salary = np.where(df['salary.gross'], mean_salary * 0.87, mean_salary)

# преобразуем numpy массив обратно в объект pandas DataFrame
mean_salary = pd.DataFrame(mean_salary, columns=['salary.temp'])

# заполняем пропущенные значения медианой
median_salary = mean_salary.median()
mean_salary = mean_salary.fillna(median_salary)

# создаем столбец salary.net
df['salary.net'] = mean_salary

In [ ]:
df.head()

In [ ]:
d_grouped

In [ ]:
# def get_description_from_api(vacancy_id):
#     """
#     Получает описание вакансии по её id через API hh.ru.
    
#     :param vacancy_id: id вакансии.
#     :return: Описание вакансии.
#     """
#     # Задаем параметры запроса.
#     params = {
#         'host': 'https://api.hh.ru',
#         'user_agent': 'api-test-agent/1.0',
#         'headers': {
#             'User-Agent': 'api-test-agent/1.0'
#         }
#     }
    
#     # Формируем URL запроса.
#     url = f"{params['host']}/vacancies/{vacancy_id}"
    
#     # Отправляем запрос к API.
#     response = requests.get(url, headers=params['headers'])
    
#     # Обрабатываем результат запроса.
#     if response.status_code == 200:
#         vacancy_data = response.json()
#         description = vacancy_data['description']
#         return description
#     else:
#         return None


# def add_description_to_dataframe(df):
#     """
#     Добавляет описание вакансии в датафрейм.
    
#     :param df: Датафрейм с вакансиями.
#     :return: Датафрейм с добавленными описаниями вакансий.
#     """
#     # Создаем новый столбец для описаний вакансий.
#     df['description'] = None
    
#     # Проходим по всем вакансиям в датафрейме.
#     for index, row in df.iterrows():
#         # Получаем id вакансии.
#         vacancy_id = row['id']
        
#         # Получаем описание вакансии.
#         description = get_description_from_api(vacancy_id)
        
#         # Записываем описание вакансии в датафрейм.
#         df.at[index, 'description'] = description
    
#     return df

In [ ]:
# # Получаем описания для каждой вакансии датафрейма, записываем результат

# def get_description(vacancy_id):
#     url = f'https://api.hh.ru/vacancies/{vacancy_id}'
#     headers = {'User-Agent': 'Mozilla/5.0'}
#     description = ""
#     while True:
#         try:
#             response = requests.get(url, headers=headers)
#             if response.ok:
#                 data = response.json()
#                 description = data['description']
#             break
#         except requests.exceptions.RequestException:
#             print(f"Ошибка получения описания вакансии {vacancy_id}. Повтор запроса через 5 секунд.")
#             time.sleep(5)
#             continue
#     return description

# # Применяем функцию для каждой вакансии в датафрейме и записываем результат в новый столбец
# df['description'] = df['id'].apply(get_description)


In [ ]:
# # Проверяем, что все значения в столбце id являются строками
# print(df['id'].apply(type).value_counts())

# # Преобразуем все значения в столбце id в строковый тип
# df['id'] = df['id'].astype(str)

# Выбираем только уникальные идентификаторы вакансий
unique_ids = df['id'].unique()
print(len(unique_ids))

# Проверяем, что количество уникальных идентификаторов вакансий совпадает с общим количеством вакансий в датафрейме
print(len(unique_ids) == len(df))


In [ ]:
# # Выбираем первые 10 вакансий и выводим их описание
# for index, row in df.head(10).iterrows():
#     print(get_description(row['id']))


In [ ]:
# get_description(79110745)

In [ ]:
# def get_description(vacancy_id):
#     url = f'https://api.hh.ru/vacancies/{vacancy_id}'
#     headers = {'User-Agent': 'Mozilla/5.0'}
#     description = ""
#     while True:
#         try:
#             response = requests.get(url, headers=headers)
#             if response.ok:
#                 data = response.json()
#                 description = data['description']
#                 break
#         except requests.exceptions.RequestException:
#             print(f"Ошибка получения описания вакансии {vacancy_id}. Повтор запроса через 5 секунд.")
#             time.sleep(5)
#             continue
#     return description


In [ ]:
# df.description.unique()

In [ ]:
df['type.name'].unique()

In [ ]:
df.loc[df['type.name'] == 'Рекламная']

In [ ]:
df.info()

In [ ]:
# # Напишем функцию, которая переведет все столбцы датасета в numeric
# # Если это столбец с датой, переводим в формат datetime

# def convert_columns(df):
#     for col in df.columns:
#         if 'date' in col.lower():
#             try:
#                 df[col] = pd.to_datetime(df[col])
#             except:
#                 pass
#         else:
#             try:
#                 df[col] = df[col].astype(float)
#             except:
#                 pass
#     return df

In [ ]:
# df = convert_columns(df)

In [ ]:
# Уберем теги из описаний вакансий при помощи ранее созданной функции

df[['description']] = df[['description']].applymap(remove_tags)

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f'Количество цифровых признаков -- {len(numeric_cols)}')
print(f'Цифровые признаки: {numeric_cols}')
print(f'Количество строковых признаков -- {len(categorical_cols)}')
print(f'Строковые признаки: {categorical_cols}')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
colours = ['#993366', '#FFFF00']
sns.heatmap(df.isnull(), cmap=sns.color_palette(colours), ax=ax)
# Decorations
plt.title('Матрица пропущенных значений набора данных', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()


In [ ]:
df.describe()


Кластеризация текстовых данных - это сложная задача машинного обучения, но можно использовать различные методы для ее решения. Вот несколько подходов:

1.Кластеризация на основе слов: вы можете использовать методы машинного обучения, такие как K-средних или DBSCAN, чтобы кластеризовать описания вакансий на основе слов в тексте. В этом случае вам нужно будет создать матрицу признаков, используя методы векторизации, такие как TF-IDF или Word2Vec, чтобы преобразовать текст в числовые значения. Затем можно использовать эти числа в качестве входных данных для алгоритма кластеризации.

2.Кластеризация на основе тем: другой подход - использовать тематическое моделирование для кластеризации описаний вакансий на основе тем, которые встречаются в тексте. Например, можно использовать методы LDA (Latent Dirichlet Allocation) или NMF (Non-Negative Matrix Factorization), чтобы найти темы, связанные с определенными профессиональными навыками или областями. Затем можно кластеризовать описания вакансий на основе этих тем.

3.Кластеризация на основе эмбеддингов: еще один подход - использовать предварительно обученные модели эмбеддингов (например, Word2Vec или GloVe), чтобы преобразовать текст в числовые векторы, представляющие значения слов. Затем можно использовать алгоритмы кластеризации, такие как DBSCAN или AgglomerativeClustering, чтобы кластеризовать описания вакансий на основе этих эмбеддингов.

Какой из этих подходов выбрать зависит от вашей задачи и доступных ресурсов. В любом случае, для успешной кластеризации необходимо иметь достаточно большой и разнообразный набор данных.

Воспользуемся Word2Vec:

In [ ]:
import gensim
from gensim.models import Word2Vec

In [ ]:
# Создадим список списков слов для обучения модели Word2Vec. Для этого необходимо разбить каждое предложение 
# на список отдельных слов и добавить каждый список в основной список.

sentences = df["snippet.requirement"].apply(lambda x: x.split() if x else [])

In [ ]:
model = Word2Vec(sentences, vector_size=300, window=5, min_count=1, workers=4)


In [ ]:
vectors = df['snippet.requirement'].apply(lambda x: model.wv[x.split()]).tolist()

In [ ]:
df['snippet.requirement'].isna().sum()

In [ ]:
df.shape

In [ ]:
df = df.dropna(subset=['snippet.requirement'])

In [ ]:
df.shape

In [ ]:
import numpy as np
vectors = np.array(df['snippet.requirement'].apply(lambda x: model.wv[x.split()]).tolist())


Для кластеризации векторов описаний вакансий можно использовать алгоритм кластеризации KMeans, который является одним из наиболее распространенных методов машинного обучения для кластеризации данных.

In [ ]:
from sklearn.cluster import KMeans

# Выберем количество кластеров
num_clusters = 10

# Инициализируем kmeans
kmeans = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=100)

# Обучаем kmeans на наших векторах
X = vectors
kmeans.fit(X)

# Получаем метки кластеров для наших векторов
cluster_labels = kmeans.labels_


In [ ]:
# Ошибка выше происходит из-за того, что массив содержит векторы разной длины
# Определим максимальную длину вектора среди всех векторов в списке:
max_len = max(len(v) for v in vectors)

# Добавим недостающие элементы в конец каждого вектора, заполнив их нулями:
for i, v in enumerate(vectors):
    vectors[i] = np.pad(v, (0, max_len - len(v)), 'constant')